# 使用GalTransl进行翻译

- **项目地址**: [https://github.com/xd2333/GalTransl](https://github.com/xd2333/GalTransl)

## 使用方法

### 导入models
导入 `fatecyx/galtransl` 模型。

### 导入datasets
导入 `fatecyx/llmserver` 数据集。

### 导入galtransl工程目录
通过datasets导入galtransl工程目录，替换以下命令中的 `/kaggle/input/dec-i18n-project2/DEC_i18n_project`：

```bash
!cp -r /kaggle/input/dec-i18n-project2/DEC_i18n_project /kaggle/working/galtransl_project
```
## 结果

- 结束时会生成 `/kaggle/working/result.zip` 文件。
- 成功结束时，日志会打印 `Process completed successfully.`。
- 如果未完成，则会显示 `Process timed out and was terminated.`。此时可以将 `result.zip` 重新作为输入文件再次运行。

In [ ]:
!cp -r /kaggle/input/dec-i18n-project2/DEC_i18n_project /kaggle/working/galtransl_project

!cp -r /kaggle/input/llmserver/LLMServer /kaggle/working/LLMServer
!cp -r /kaggle/input/galtransl/GalTransl /kaggle/working/GalTransl

%cd /kaggle/working/GalTransl
!pip install virtualenv
!virtualenv env
!/kaggle/working/GalTransl/env/bin/pip install -r requirements.txt
!/kaggle/working/GalTransl/env/bin/pip install opencc==1.1.6

%cd /kaggle/working/LLMServer
!pip install "diskcache>=5.6.1"
!pip install llama-cpp-python -i https://sakurallm.github.io/llama-cpp-python/whl/cu121
!pip install -q -r requirements.llamacpp.txt
!pip install -q pyngrok

In [ ]:
# ngrok：本地HTTP服务用于中途下载（一般用不上）
ngrokToken = ""

PORT = 8000
DIRECTORY = "/kaggle/working"
def local_http():
    import http.server
    import socketserver

    Handler = http.server.SimpleHTTPRequestHandler
    Handler.directory = DIRECTORY

    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print("HTTP server is running at port", PORT)
        print("Server directory is", DIRECTORY)
        httpd.serve_forever()

if ngrokToken:
    threading.Thread(target=local_http, daemon=True, args=()).start()
        
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(PORT)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)


In [ ]:
# 启动sakura模型
%cd /kaggle/working/LLMServer
from huggingface_hub import hf_hub_download
from pathlib import Path
MODEL_PATH = "/kaggle/input/galtransl/gguf/galtransl-7b-v2-q6_k/1/GalTransl-7B-v2-Q6_K.gguf"

import subprocess
import threading
import time

def local_model(dic_status):
    cmd = f"python server.py --model_name_or_path {MODEL_PATH} --llama_cpp --use_gpu --model_version 0.9 --trust_remote_code --no-auth"
    
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    for line in p.stderr:
        if not dic_status['status']:
            s2 = line.decode()
            dic_status['msg'] = s2
            if "INFO Running on http://127.0.0.1:5000" in s2:
                dic_status['status'] = True
        #print(line.decode(), end='')
dic_status = {'status': False, 'msg': ''}

threading.Thread(target=local_model, daemon=True, args=(dic_status,)).start()

s = ""
while not dic_status['status']:
    if s != dic_status['msg']:
        s = dic_status['msg']
        print(s)
print("模型启动成功")


try:
    result = subprocess.run(
        ["/kaggle/working/GalTransl/env/bin/python", 
        "-m", "GalTransl",
        "-p", "/kaggle/working/galtransl_project",
        "-t" "galtransl-v2"],
        cwd="/kaggle/working/GalTransl",
        stdout=subprocess.DEVNULL,  # 屏蔽标准输出
        stderr=subprocess.DEVNULL,  # 屏蔽错误输出
        timeout=int(11.5*3600)
    )
    print("Process completed successfully.")
except subprocess.TimeoutExpired:
    print("Process timed out and was terminated.")
    
# %cd /kaggle/working/GalTransl
# !/kaggle/working/GalTransl/env/bin/python -m GalTransl -p /kaggle/working/galtransl_project -t galtransl-v2

import shutil
shutil.make_archive('/kaggle/working/result', 'zip', '/kaggle/working/galtransl_project')